In [1]:
import os

from openeye import oechem, oespruce

In [2]:
pdb_fn = "input_pdbs/pdb8dgy.ent"
ref_fn = "input_pdbs/reference.pdb"
loop_db = "rcsb_spruce.loop_db"

out_dir = "design_units/"

In [3]:
## Load molecule to extract
ifs = oechem.oemolistream()
ifs.SetFlavor(
    oechem.OEFormat_PDB,
    oechem.OEIFlavor_PDB_Default | oechem.OEIFlavor_PDB_DATA,
)
ifs.open(pdb_fn)
in_mol = oechem.OEGraphMol()
oechem.OEReadMolecule(ifs, in_mol)
ifs.close()

## Load reference molecule
ifs.open(ref_fn)
ref_mol = oechem.OEGraphMol()
oechem.OEReadMolecule(ifs, ref_mol)
ifs.close()

In [ ]:
## First convert ref from mol into design unit
# opts = oespruce.OEMakeDesignUnitOptions()
# ref_du = oespruce.OEMakeDesignUnits(
#     ref_mol, oespruce.OEStructureMetadata(), opts
# )
# ref_du = next(iter(ref_du))

# ref_du = oechem.OEDesignUnit()
# oespruce.OEMakeDesignUnit(ref_du, ref_mol, oechem.OEGraphMol())

In [4]:
## Make BioUnitExtractionOptions object and set superpose
##  (aligning to reference)
bio_opts = oespruce.OEBioUnitExtractionOptions()
bio_opts.SetSuperpose(True)

biounits = oespruce.OEExtractBioUnits(in_mol, ref_mol, bio_opts)
aligned_prot = list(biounits)[0]

  1 bio mols found


In [5]:
## Make DesignUnitOptions
opts = oespruce.OEMakeDesignUnitOptions()
# opts.SetBioUnitExtractionOptions(bio_opts)
opts.GetPrepOptions().GetBuildOptions().GetLoopBuilderOptions().SetBuildTails(
    False
)
opts.GetPrepOptions().GetBuildOptions().GetLoopBuilderOptions().SetLoopDBFilename(
    loop_db
)
opts.GetPrepOptions().GetEnumerateSitesOptions().SetRestrictToRefSite(True)

## Make Metadata object
metadata = oespruce.OEStructureMetadata()

In [6]:
design_units = oespruce.OEMakeDesignUnits(aligned_prot, metadata, opts)
design_units = list(design_units)

DPI: 0.09, RFree: 0.20, Resolution: 1.65
Processing BU # 1 with title: 3C-LIKE PROTEINASE, chains AB
Found unresolved N-terminal with 5 residues before HIS -1   A 1  , with sequence MHHHH
Found 4 residue gap between HIS 71   A 1   and ALA 76   A 1  , with sequence IGAP
Found unresolved C-terminal with 2 residues after VAL 304   A 1  , with sequence MQ
Attempting to build loop: IGAP between HIS 71   A 1   and ALA 76   A 1  
Opened database rcsb_spruce.loop_db
LoopDatabase Info: 
    161412 loops from RCSB last synced on 04-10-2020, were added to LoopTemplateDatabase on 04-22-2020 using Spruce Toolkit 1.1.0.a.1
    The loop database was built with a max loop length of 22, a terminus crop length of 2. Regular secondary structures were excluded
Superposition - RMSD: 0.05, Ref: AB, Fit: BA, SeqScore: 3010


In [8]:
out_base = (
    f"{out_dir}/{os.path.splitext(os.path.basename(pdb_fn))[0]}_{{}}.oedu"
)

design_units = list(design_units)
# for i, du in enumerate(design_units):
#     print(i)
#     oechem.OEWriteDesignUnit(out_base.format(i), du)

In [34]:
print(design_units)
lig_mol = oechem.OEGraphMol()
prot_mol = oechem.OEGraphMol()
design_units[0].GetLigand(lig_mol)
design_units[0].GetProtein(prot_mol)
print(design_units[0].GetSiteResidues())
du_site_res = sorted(
    [int(r.split(":")[1]) for r in design_units[0].GetSiteResidues()]
)

[<oechem.OEDesignUnit; proxy of <Swig Object of type 'std::vector< OEBio::OEDesignUnit * >::value_type' at 0x7f5e6633bb40> >, <oechem.OEDesignUnit; proxy of <Swig Object of type 'std::vector< OEBio::OEDesignUnit * >::value_type' at 0x7f5e6633b5d0> >]
('MET:25: :B:1: ', 'LEU:27: :B:1: ', 'PRO:39: :B:1: ', 'HIS:41: :B:1: ', 'LEU:49: :B:1: ', 'PRO:52: :B:1: ', 'TYR:54: :B:1: ', 'PHE:143: :B:1: ', 'LEU:144: :B:1: ', 'CYS:145: :B:1: ', 'GLY:146: :B:1: ', 'SER:147: :B:1: ', 'CYS:148: :B:1: ', 'GLY:149: :B:1: ', 'HIS:166: :B:1: ', 'GLN:167: :B:1: ', 'MET:168: :B:1: ', 'GLU:169: :B:1: ', 'LEU:170: :B:1: ', 'ALA:171: :B:1: ', 'HIS:175: :B:1: ', 'MET:189: :B:1: ', 'ASP:190: :B:1: ', 'LYS:191: :B:1: ', 'GLN:192: :B:1: ', 'VAL:193: :B:1: ', 'HIS:194: :B:1: ', 'GLN:195: :B:1: ', 'THR:26: :B:1: ', 'ASN:28: :B:1: ', 'ARG:40: :B:1: ', 'ALA:46: :B:1: ', 'TYR:121: :B:1: ', 'GLY:141: :B:1: ', 'SER:150: :B:1: ', 'TYR:164: :B:1: ', 'GLY:173: :B:1: ', 'THR:174: :B:1: ', 'THR:176: :B:1: ')


In [36]:
for a in lig_mol.GetAtoms():
    print(oechem.OEGetResiduea)

 0 O
 1 C
 2 N
 3 C
 4 C
 5 C
 6 C
 7 C
 8 C
 9 N
10 C
11 C
12 C
13 C
14 N
15 C
16 C
17 O
18 C
19 O
20 O
21 O
22 C
23 C
24 C
25 C
26 C
27 C
28 C
29 C
30 C
31 C
32 C
33 H
34 H
35 H
36 H
37 H
38 H
39 H
40 H
41 H
42 H
43 H
44 H
45 H
46 H
47 H
48 H
49 H
50 H
51 H
52 H
53 H
54 H
55 H
56 H
57 H
58 H
59 H
60 H
61 H
62 H
63 H
64 H
65 H
66 H
67 H
68 H
69 H
70 H
71 H
72 H
73  


In [ ]:
out_base = f"{out_dir}/{os.path.splitext(os.path.basename(pdb_fn))[0]}_du_protein_{{}}.pdb"
ofs = oechem.oemolostream()
ofs.SetFlavor(
    oechem.OEFormat_PDB,
    oechem.OEOFlavor_PDB_Default,
)
prot_mol = oechem.OEGraphMol()
for i, du in enumerate(design_units):
    print(out_base.format(i))

    prot_mol.Clear()
    du.GetProtein(prot_mol)

    ofs.open(out_base.format(i))
    oechem.OEWriteMolecule(ofs, prot_mol)
    ofs.close()

In [10]:
import re

import yaml

active_site_fn = "mers_active_site.yaml"
active_site_dict = yaml.load(open(active_site_fn), Loader=yaml.CLoader)

## First parse all resids from the atom selection texts
pat_match = r"^resi ((?:[0-9]+\+)*[0-9]+)$"
matches = [re.match(pat_match, sel) for sel in active_site_dict.values()]
matches = [m.groups()[0] for m in matches if m is not None]
all_active_res = {int(resid) for m in matches for resid in m.split("+")}
print(sorted(all_active_res))

## Format in the site residue format for OE
res_sites = []
hv = oechem.OEHierView(aligned_prot)
for chain in hv.GetChains():
    chain_id = chain.GetChainID()
    for frag in chain.GetFragments():
        for hres in frag.GetResidues():
            if hres.GetResidueNumber() not in all_active_res:
                continue
            site_spec = f"{hres.GetResidueName()}:{hres.GetResidueNumber()}: :{chain_id}"
            res_sites.append(site_spec)

print(res_sites)

[25, 26, 27, 41, 49, 54, 143, 144, 145, 146, 147, 148, 166, 167, 168, 169, 170, 171, 172, 175, 184, 189, 191, 192, 193, 194, 195]
['MET:25: :A', 'THR:26: :A', 'LEU:27: :A', 'HIS:41: :A', 'LEU:49: :A', 'TYR:54: :A', 'PHE:143: :A', 'LEU:144: :A', 'CYS:145: :A', 'GLY:146: :A', 'SER:147: :A', 'CYS:148: :A', 'HIS:166: :A', 'GLN:167: :A', 'MET:168: :A', 'GLU:169: :A', 'LEU:170: :A', 'ALA:171: :A', 'ASN:172: :A', 'HIS:175: :A', 'MET:184: :A', 'MET:189: :A', 'LYS:191: :A', 'GLN:192: :A', 'VAL:193: :A', 'HIS:194: :A', 'GLN:195: :A']


In [32]:
du = oechem.OEDesignUnit(aligned_prot, res_sites)
prot_mol = oechem.OEGraphMol()
lig_mol = oechem.OEGraphMol()
print(du.GetProtein(prot_mol))
print(du.GetLigand(lig_mol))

print(prot_mol)
lig_res = []
for r in oechem.OEGetResidues(prot_mol):
    if r.GetName() == "S9U":
        lig_res.append(r)
print(lig_res)
# oespruce.OEMakeDesignUnit(du, aligned_prot, oechem.OEGraphMol())

# oespruce.OEMakeDesignUnits(aligned_prot, )
# dir(aligned_prot)

True
False
<oechem.OEGraphMol; proxy of <Swig Object of type 'OEGraphMolWrapper *' at 0x7f5e451ef660> >
[<oechem.OEResidue; proxy of <Swig Object of type 'OEChem::OEResidue *' at 0x7f5e5861b7b0> >]


# For already generated DesignUnits

In [9]:
du_dir = "design_units/"
frag_dir = "fragalysis/"

In [19]:
du = oechem.OEDesignUnit()
oechem.OEReadDesignUnit(f"{du_dir}/pdb8dgy_0.oedu", du)

True

In [20]:
lig_mol = oechem.OEGraphMol()
prot_mol = oechem.OEGraphMol()
print(du.GetLigand(lig_mol))
print(du.GetProtein(prot_mol))

print(lig_mol.GetCoords())

True
True
{0: (8.736869812011719, 1.840950608253479, 20.722593307495117), 1: (8.991327285766602, 1.8149514198303223, 21.915855407714844), 2: (9.627471923828125, 0.8253679275512695, 22.621824264526367), 3: (9.883052825927734, -0.4560964107513428, 21.951887130737305), 4: (10.668458938598633, -1.3675775527954102, 22.87837791442871), 5: (12.103649139404297, -0.9021434783935547, 23.11640739440918), 6: (12.891031265258789, -0.7986127138137817, 21.81521987915039), 7: (12.858545303344727, -1.8402495384216309, 24.04183578491211), 8: (8.529062271118164, -1.0746678113937378, 21.6446590423584), 9: (8.316762924194336, -1.6040165424346924, 20.41315460205078), 10: (7.0836944580078125, -2.1928112506866455, 19.99717140197754), 11: (6.5677032470703125, -1.4264198541641235, 18.764925003051758), 12: (6.186033248901367, 0.010240626521408558, 19.11873435974121), 13: (5.628711700439453, 0.7831991910934448, 17.94416618347168), 14: (4.517642974853516, 1.4991313219070435, 18.325023651123047), 15: (4.19306564331

In [21]:
## Load a random molecule from SDF file
sdf_fn = f"{frag_dir}/aligned/Mpro-P2007_0A/Mpro-P2007_0A.sdf"
ifs = oechem.oemolistream()
ifs.SetFlavor(
    oechem.OEFormat_SDF,
    oechem.OEIFlavor_SDF_Default,
)
ifs.open(sdf_fn)
new_lig = oechem.OEGraphMol()
oechem.OEReadMolecule(ifs, new_lig)
ifs.close()

print(new_lig.GetCoords())

{0: (9.133999824523926, 1.0069999694824219, 21.385000228881836), 1: (8.734000205993652, -0.08100000023841858, 21.79800033569336), 2: (9.326000213623047, -0.7480000257492065, 23.024999618530273), 3: (10.805000305175781, -0.45899999141693115, 23.16900062561035), 4: (11.255999565124512, 0.5249999761581421, 24.040000915527344), 5: (12.58899974822998, 0.9039999842643738, 24.055999755859375), 6: (13.498000144958496, 0.29899999499320984, 23.201000213623047), 7: (11.72700023651123, -1.100000023841858, 22.344999313354492), 8: (13.053000450134277, -0.7110000252723694, 22.3700008392334), 9: (14.196000099182129, -1.5379999876022339, 21.347000122070312), 10: (7.703999996185303, -0.7609999775886536, 21.211000442504883), 11: (7.065999984741211, -0.328000009059906, 20.047000885009766), 12: (5.927999973297119, 0.4569999873638153, 20.079999923706055), 13: (5.631999969482422, 0.7609999775886536, 18.75), 14: (6.514999866485596, 0.19699999690055847, 17.92300033569336), 15: (7.394000053405762, -0.4729999899

In [23]:
print(oespruce.OEMakeDesignUnit(du, prot_mol, new_lig))
print(du.HasLigand(), du.HasProtein())

True
False False


In [15]:
## Try and save full design unit as PDB?
out_fn = "test/pdb8dgy_0_P2007_0A_lig.pdb"
ofs = oechem.oemolostream()
ofs.SetFlavor(
    oechem.OEFormat_PDB,
    oechem.OEOFlavor_PDB_Default,
)
ofs.open(out_fn)
oechem.OEWriteDesignUnit(ofs, du)
ofs.close()

TypeError: Wrong number or type of arguments for overloaded function 'OEWriteDesignUnit'.
  Possible C/C++ prototypes are:
    OEBio::OEWriteDesignUnit(std::string const &,OEBio::OEDesignUnit const &)
    OEBio::OEWriteDesignUnit(OEPlatform::oeostream &,OEBio::OEDesignUnit const &)


In [18]:
## Doesn't work... maybe we can combine the prot and lig molecules?
prot_mol.Clear()
lig_mol.Clear()
print(du)
print(du.GetProtein(prot_mol))
print(du.GetLigand(lig_mol))

<oechem.OEDesignUnit; proxy of <Swig Object of type 'std::vector< OEBio::OEDesignUnit * >::value_type' at 0x7f0f540f0d80> >
False
False
